Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор размеченных текстов.

## Оглавление:
* [1. Подготовка](#1)
* [2. Обучение](#2)
* [3. Выводы](#3)
* [Чек-лист проверки](#4)

# 1. Подготовка <a class="anchor" id="1"></a>

Импортируем библиотеки:

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

# <импорт библиотеки sklearn>
import sklearn

# <Отключение предупреждений>
import warnings
warnings.filterwarnings('ignore')

# <импорт библиотеки numpy>
import numpy as np

# <импорт библиотеки tqdm, позволяющей отслеживать прогресс>
from tqdm import notebook

Прочитаем файл с данными:

In [2]:
# <чтение файла с данными с сохранением в переменную df>
df = pd.read_csv('/datasets/toxic_comments.csv')

Осмотрим данные:

In [3]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

Понизим регистр текста:

In [4]:
df['text'] = df['text'].astype(str).str.lower()

Проведем лемматизацию текста:

In [6]:
# <импорт библиотеки nltk и ее лемматизатора>
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize

# <импорт регулярных выражений>
import re

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
# <создадим переменную корпуса текстов>
corpus= df['text'].values

Создадим функцию лемматизации и функцию очистки текста:

In [8]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    s = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in s])

In [9]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    return " ".join(text.split())

Проведем лемматизацию:

In [66]:
%%time
for i in range(len(df['text'])):
    corpus[i] = lemmatize(clear_text(corpus[i]))

Wall time: 2min 2s


Для модели *BERT* сделаем свой датафрейм где текст только очистим:

In [11]:
%%time
corpus1 = df['text'].values
for i in range(len(df['text'])):
    corpus1[i] = clear_text(corpus1[i])
    
df1 = df
df1['text'] = pd.Series(corpus1)

Wall time: 4.39 s


Датафрейм ниже для модели которая будет использовать *TF-IDF* матрицу как признаки.

In [67]:
df['text'] = pd.Series(corpus)

Разделим данные на обучающую и тестовую выборку (мы будем использовать кросс-валидацию для оценки моделей):

In [13]:
from sklearn.model_selection import train_test_split

In [68]:
train, test = train_test_split(df, test_size=0.2, random_state=12345)

### Вывод

Мы осмотрели данные, провели лемматизацию и очистку текста. В следующем пункте проведем апсэмплинг, создадим матрицу *TF-IDF* и попробуем обучить на ней модель. Также попробуем обучить другие модели на эмбеддингах BERT.

# 2. Обучение <a class="anchor" id="2"></a>

Проведем апсэмплинг.

In [15]:
print("0:",train[train.toxic==0]['text'].count())
print("1:",train[train.toxic==1]['text'].count())

0: 114670
1: 12986


In [16]:
from sklearn.utils import resample

In [69]:
train_majority = train[train.toxic==0]
train_minority = train[train.toxic==1]
 
# <апсэмплинг минорного класса>
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=114670,     
                                 random_state=123) 
 
# <соединим минорный класс с мажоритарным>
train_upsampled = pd.concat([train_minority_upsampled, train_majority])

train_upsampled.toxic.value_counts()

1    114670
0    114670
Name: toxic, dtype: int64

Выделим признаки и целевой признак в обучающей и тестовой выборке.

In [70]:
features_train = train_upsampled['text']
target_train = train_upsampled['toxic']
features_test = test['text']
target_test = test['toxic']

Очистим текст от стоп-слов и создадим матрицу *TF-IDF*:

In [71]:
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [73]:
features_train_tf = count_tf_idf.fit_transform(features_train)
features_test_tf = count_tf_idf.transform(features_test)

Попробуем обучить логистическую регрессию на матрице *TF-IDF*:

In [22]:
# <Импортируем метод логистической регрессии>
from sklearn.linear_model import LogisticRegression
# <Импортируем функцию cross_val_score>
from sklearn.model_selection import cross_val_score

In [23]:
%%time
# <Создадим модель лог. регрессии>
model_lr = LogisticRegression(class_weight = 'balanced', solver='liblinear', random_state=12345)

# <Оценим качество модели, обученной в ходе перекрестной проверки>
score = cross_val_score(model_lr,features_train_tf, target_train,cv=4,scoring='f1').mean()

# <обучаем модель> 
model_lr.fit(features_train_tf, target_train)

# <выведем долю правильных ответов>
score

Wall time: 11.4 s


0.9583704953694141

Проверим переменные в памяти:

In [24]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

               train_upsampled: 88.7 MiB
                features_train: 86.9 MiB
                            df: 66.0 MiB
                           df1: 66.0 MiB
                         train: 53.9 MiB
                train_majority: 49.5 MiB
      train_minority_upsampled: 39.2 MiB
                          test: 13.3 MiB
                 features_test: 13.1 MiB
                train_minority:  4.5 MiB


In [25]:
del train_upsampled
del features_train
del train
del train_majority
del train_minority_upsampled
del features_test
del train_minority
del test
del df

Попробуем использовать *BERT*. Очистим текст от стоп-слов:

In [26]:
df1['text'] = df1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

Далее импортируем библиотеки, используем видеокарту для вычислений. Создадим эмбеддинги.

In [27]:
import torch
import transformers

In [28]:
device = torch.device("cuda")

In [29]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased').to(device)

In [30]:
sample = df1.sample(30000).reset_index(drop=True)

In [31]:
%%time
tokenized = sample['text'].apply(
    lambda x: tokenizer.encode(x[:512], add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Wall time: 26.5 s


In [32]:
# <с большим batch size сталкивался с недостатком памяти видеокарты>
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

Создадим датафрейм, где эмбеддинги - признаки, также там будет присутствовать целевой признак.

In [33]:
feature = np.concatenate(embeddings)
df_bert = pd.DataFrame(feature)
df_bert['toxic'] = sample['toxic']
df_bert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Columns: 769 entries, 0 to toxic
dtypes: float32(768), int64(1)
memory usage: 88.1 MB


Разделим выборки на обучающую и тестовую:

In [34]:
train_bert, test_bert = train_test_split(df_bert, test_size=0.2, random_state=12345)

Проведем апсэмплиннг. Посчитаем количество текстов в кажом классе:

In [35]:
print("0:",train_bert[train_bert.toxic==0]['toxic'].count())
print("1:",train_bert[train_bert.toxic==1]['toxic'].count())

0: 21517
1: 2483


In [36]:
train_majority = train_bert[train_bert.toxic==0]
train_minority = train_bert[train_bert.toxic==1]
 
# <даунсэмплинг мажоритарного класса>
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=21517,     
                                 random_state=123) 
 
# <соединим минорный класс с мажоритарным>
train_upsampled = pd.concat([train_minority_upsampled, train_majority])

train_upsampled.toxic.value_counts()

1    21517
0    21517
Name: toxic, dtype: int64

Разделим наши выборки на признаки и целевой признак:

In [37]:
features_train_bert = train_upsampled.drop('toxic', axis=1)
target_train_bert = train_upsampled['toxic']
features_test_bert = test_bert.drop('toxic', axis=1)
target_test_bert = test_bert['toxic']

Удалим ненужные переменные:

In [38]:
del test_bert
del train_bert
del df_bert
del train_upsampled
del train_majority
del train_minority
del train_minority_upsampled

Обучим модель логистической регрессии:

In [39]:
%%time
# <Создадим модель лог. регрессии>
model_lr_bert = LogisticRegression(solver='liblinear', random_state=12345)

# <Оценим качество модели, обученной в ходе перекрестной проверки>
score = cross_val_score(model_lr_bert,features_train_bert, target_train_bert,cv=4,scoring='f1').mean()

# <обучаем модель> 
model_lr_bert.fit(features_train_bert, target_train_bert)

# <выведем долю правильных ответов>
score

Wall time: 1min 38s


0.8892766207337759

Попробуем модель стохастического градиентного спуска:

In [42]:
from sklearn.linear_model import SGDClassifier

In [50]:
%%time
# <Создадим модель лог. регрессии>
model_sgd = SGDClassifier(max_iter = 1000, n_jobs=-1)

# <Оценим качество модели, обученной в ходе перекрестной проверки>
score = cross_val_score(model_sgd,features_train_bert, target_train_bert,cv=4,scoring='f1').mean()

# <обучаем модель> 
model_sgd.fit(features_train_bert, target_train_bert)

# <выведем долю правильных ответов>
score

Wall time: 17 s


0.8862954458101681

Попробуем использовать библиотеку *LightGBM* и ее *Dropouts meet Multiple Additive Regression Trees*:

In [52]:
import lightgbm as lgb

In [61]:
%%time
hyper_params = {
    'learning_rate': 0.1,
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    "num_iterations": 100,
    'max_depth': 10,
    'num_leaves': 75
}
dart = lgb.LGBMClassifier(**hyper_params)
dart.fit(features_train_bert, target_train_bert,
        eval_set=[(features_train_bert, target_train_bert)],
        eval_metric='binary_error',
        early_stopping_rounds=10, verbose = 10)

[10]	training's binary_error: 0.0738253	training's binary_logloss: 0.410284
[20]	training's binary_error: 0.0501464	training's binary_logloss: 0.281366
[30]	training's binary_error: 0.033764	training's binary_logloss: 0.214823
[40]	training's binary_error: 0.0253985	training's binary_logloss: 0.182563
[50]	training's binary_error: 0.022703	training's binary_logloss: 0.177504
[60]	training's binary_error: 0.0183111	training's binary_logloss: 0.159318
[70]	training's binary_error: 0.0159176	training's binary_logloss: 0.147786
[80]	training's binary_error: 0.0141284	training's binary_logloss: 0.13437
[90]	training's binary_error: 0.0131292	training's binary_logloss: 0.137252
[100]	training's binary_error: 0.0118279	training's binary_logloss: 0.127278
Wall time: 47.5 s


LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_iterations=100, num_leaves=75,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, task='train')

### Вывод

В следующем пункте будет видно, что вышли не самые хорошие результаты, скорее всего это связано с предобработкой данных. Хотя делал вроде все как надо. В целом было бы интересно узнать как получить на эмбеддингах *BERT* результаты лучше чем на матрице *TF-IDF*.

# 3. Выводы <a class="anchor" id="3"></a>

Лучший результат получился у модели логистической регрессии, где признаки - матрица *TF-IDF*.

In [77]:
predictions = model_lr.predict(features_test_tf)
from sklearn.metrics import f1_score
f1_score(target_test,predictions)

0.7497258771929824

У логистической регрессии на эмбеддингах *BERT* вышла худшая оценка, не совсем понятно почему.

In [41]:
predictions = model_lr_bert.predict(features_test_bert)
from sklearn.metrics import f1_score
f1_score(target_test_bert,predictions)

0.5786802030456852

Модель стохастического градиентного спуска на эмбеддингах *BERT*:

In [51]:
predictions = model_sgd.predict(features_test_bert)
from sklearn.metrics import f1_score
f1_score(target_test_bert,predictions)

0.6325340246273493

Модель *Dropouts meet Multiple Additive Regression Trees* на эмбеддингах *BERT*:

In [62]:
predictions = dart.predict(features_test_bert)
from sklearn.metrics import f1_score
f1_score(target_test_bert,predictions)

0.6428571428571429

### Вывод

Получены не самые утешительные результаты. Есть предположение что стоит добавить еще один признак - размер фразы. Или какой либо еще.
Здорово было узнать что видеокарта может здорово ускорить вычисление эмбеддингов.